In [53]:
class Word:
    def __init__(self, word):
        self.spamCount = 0
        self.hamCount = 0
        self.word = word
        self.MI = 0
        
    def addSpamCount(self):
        self.spamCount+=1
        
    def addHamCount(self):
        self.hamCount+=1    
    
    def setMI(self, mi):
        self.MI = mi
    
class wordList:
    def __init__(self):
        self.wordList = []
        self.probSpamList = []
        self.probLegitList = []
        
    def updateWordList(self, word, doctype):
        for currWord in self.wordList:
            if word == currWord.word and doctype == "ham":
                currWord.addHamCount()
                return 0
            elif word == currWord.word and doctype == "spam":
                currWord.addSpamCount()
                return 0
                
        newWord = Word(word)
        if doctype == "ham":
            newWord.addHamCount()
        elif doctype == "spam":
            newWord.addSpamCount()
        self.wordList.append(newWord)
        return 1
            
    def printWordList(self):
        for word in self.wordList:
            cnt = 0
            print(word.word + " " + "spam" + " " + str(word.spamCount))
            print(word.word + " " + "ham" + " " + str(word.hamCount))
            cnt += word.spamCount + word.hamCount
            print(str(cnt))

In [54]:
import glob
import errno

numSpam = 0
numLegit = 0
numTotal = 0
wordList = wordList()

print("HELLO")
for i in range (1,2):
    if(i != 0):
        path = "emails/emails/lemm_stop/part" + str(i) + "/*.txt"
        files = glob.glob(path)
        print(i)

        for name in files:
            checkedWordList = []
            if "spmsg" in name:
                numSpam += 1
            else:
                numLegit += 1
            numTotal += 1
            with open(name, 'r') as f:
                    for line in f:
                        for word in line.split():
                            if "spmsg" in name and word not in checkedWordList:
                                wordList.updateWordList(word, "spam")
                                checkedWordList.append(word)
                            elif word not in checkedWordList:
                                wordList.updateWordList(word, "ham")
                                checkedWordList.append(word)

wordList.printWordList()
print(str(numTotal))

HELLO
1
Subject: spam 48
Subject: ham 241
289
re spam 9
re ham 64
73
: spam 43
: ham 198
241
2 spam 22
2 ham 46
68
. spam 47
. ham 235
282
882 spam 0
882 ham 2
2
s spam 22
s ham 50
72
- spam 37
- ham 158
195
> spam 9
> ham 44
53
np spam 0
np ham 6
6
deat spam 5
deat ham 6
11
sun spam 0
sun ham 4
4
, spam 44
, ham 228
272
15 spam 11
15 ham 20
31
dec spam 1
dec ham 1
2
91 spam 1
91 ham 1
2
25 spam 7
25 ham 9
16
est spam 0
est ham 4
4
michael spam 0
michael ham 21
21
< spam 1
< ham 25
26
mmorse spam 0
mmorse ham 1
1
@ spam 15
@ ham 121
136
vm1 spam 0
vm1 ham 1
1
yorku spam 0
yorku ham 2
2
ca spam 7
ca ham 14
21
subject spam 14
subject ham 30
44
864 spam 0
864 ham 1
1
query spam 0
query ham 43
43
wlodek spam 0
wlodek ham 1
1
zadrozny spam 0
zadrozny ham 1
1
ask spam 11
ask ham 38
49
" spam 27
" ham 112
139
anything spam 5
anything ham 18
23
interest spam 13
interest ham 81
94
construction spam 0
construction ham 11
11
second spam 1
second ham 34
35
much spam 20
much ham 44
64
relate spam 3

In [55]:
import math

for word in wordList.wordList:
    try:    
        mInformation = (word.spamCount/numTotal * math.log(numTotal * word.spamCount/((word.spamCount + word.hamCount) * numSpam),2))
    except:
        mInformation = 0
    try:
        mInformation += ((numSpam - word.spamCount)/numTotal * math.log(numTotal * (numSpam - word.spamCount)/((numSpam - word.spamCount + word.hamCount) * numSpam),2))
    except:
        mInformation += 0
    try:    
        mInformation += (word.hamCount/numTotal * math.log(numTotal * word.hamCount/((word.spamCount + word.hamCount) * (word.hamCount + (numLegit))),2))
    except:
        mInformation += 0
    try:
        mInformation += ((numLegit - word.hamCount) / numTotal * math.log((numTotal * (numLegit - word.hamCount)) / ((word.hamCount + word.spamCount) * (numLegit)),2))
    except:
        mInformation += 0
    #add MI
    word.setMI(mInformation)
    
wordList.wordList.sort(key=lambda x: x.MI, reverse=True)

In [56]:
#Training
spamPrior = numSpam / numTotal
legitPrior = numLegit / numTotal

for word in wordList.wordList:
    print(word.word + " " + str(word.MI))
    probSpam = (numSpam + 1) / (word.spamCount + 2)
    probLegit = (numLegit + 1) / (word.hamCount + 2)
    wordList.probSpamList.append(probSpam)
    wordList.probLegitList.append(probLegit)
    
print(wordList.probSpamList)
print(wordList.probLegitList)

owner-operate 7.247319454007201
counter 7.247319454007201
compensaation 7.247319454007201
703-5390 7.247319454007201
b2998 7.247319454007201
auto 7.247319454007201
amazingly 7.247319454007201
cheaper 7.247319454007201
lowest 7.247319454007201
pva 7.247319454007201
1257 7.247319454007201
kenmore 7.247319454007201
90029 7.247319454007201
mechanic 7.247319454007201
economical 7.247319454007201
felton 7.247319454007201
wild 7.247319454007201
cook 7.247319454007201
wood 7.247319454007201
noon 7.247319454007201
cozy 7.247319454007201
season 7.247319454007201
muzzel 7.247319454007201
loader 7.247319454007201
deer 7.247319454007201
archery 7.247319454007201
oct 7.247319454007201
turkey 7.247319454007201
sesson 7.247319454007201
110734 7.247319454007201
2622 7.247319454007201
iq 7.247319454007201
zsazsa36 7.247319454007201
sparkle124 7.247319454007201
173 7.247319454007201
herehttp 7.247319454007201
bizman 7.247319454007201
tj 7.247319454007201
amcap 7.247319454007201
htmlor 7.247319454007201
p

In [9]:
import math

#function returns the prediction of the system whether spam 
def predict(numSpam, numLegit,spamWords,legitWords):
    #numSpam contains total number of spam
    #numLegit contains total number of legit
    #spamWords contains the probabilities of the words as spam
    #legitWords contains the probabilities of the words as legit
    
    total = probabilityOfSpam(numSpam,numLegit,spamWords) - probabilityOfLegit(numSpam,numLegit,legitWords)
    
    if total >= math.log(999,10):
        return "spam"
    else:
        return "legitimate"
    

#Probability of spam, words is the vector of SIGNIFICANT words in the list obtained in the MI phase
def probabilityOfSpam(numSpam, numLegit, probabilities):
    #numSpam contains total number of spam
    #numLegit contains total number of legit
    #*words contains all the values of the probability of each signifacant word
    
    probSpam = math.log((numSpam/(numSpam+numLegit)),10) #the probability of getting a spam document
    accumulatedProbabilityOfWords = 1
    for num in list(probabilities):
        accumulatedProbabilities *= num
    
    accumulatedProbabilities = math.log(accumulatedProbabilities)
    return probSpam + accumulatedProbabilities

#Probability that the document is legitimate
def probabilityOfLegit(numSpam, numLegit, probabilities):
    #numSpam contains total number of spam
    #numLegit contains total number of legit
    #*words contains all the values of the probability of each signifacant word
    
    probLegit = math.log(numLegit/(numSpam+numLegit),10) #the probability of getting a legit document
    accumulatedProbabilities = 1
    
    for num in list(probabilities):
        accumulatedProbabilities *= num
    
    accumulatedProbabilites = math.log(accumulatedProbabilities)
    return probLegit + accumulatedProbabilities


# An Evaluation of Naive Bayesian Anti-Spam Filtering
**Authors**: Ion Androutsopoulos, John Koutsias, Konstantinos V. Chandrinos, George Paliouras and Constantine D. Spyropoulos

**Recreated by** : Amadora, Angelo and Choy, Seaver Mathew

---
# 1 Introduction

Unsolicited bulk e-mail which are sent to thousands  of recipients, is becoming common. Anyone with an e-mail will most likely have encountered or at least been sent these e-mails called "spam". Spam is annoying and sent by marketers in order to entice unsuspecting people into clicking links for many purposes unknown to them. Apart from wasting time, spam costs money to users
with dial-up connections, wastes bandwidth, and may expose under-aged recipients to unsuitable (e.g.
pornographic) content. 


There are many anti-spam filters available however, they rely mostly on manually constructed patternmatching. These systems need to be maintained in order to match a user's incoming message. This method requires both time and expertise. It would stand to reason that a system that would learn automatically to separate spam from legitimate massages would present significant changes.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline #for jupyter only... this will not work in pycharm
#Creating a Table

def table_row_generator(index,pd,label_type,l,a,avg_recall,avg_precision,avg_accuracy,avg_accuracy_base,avg_tcr):
    raw_data = {
        '#': index,
        'Filter Configuration': [label_type],
        'Lambda': [l], 
        'No. of attrib.': [a],
        'Spam Recall': [avg_recall],
        'Spam Precision': [avg_precision],
        'Weighted Accuracy': [avg_accuracy],
        'Baseline W. Acc': [avg_accuracy_base],
        'TCR': [avg_tcr]
    }
    return pd.DataFrame(raw_data, columns = ['#', 'Filter Configuration', 'Lambda', 'No. of attrib.', 'Spam Recall', 'Spam Precision', 'Weighted Accuracy', 'Baseline W. Acc', 'TCR'])
       
df_i = table_row_generator(1,pd,label_type[0],1,50,avg_recall,avg_precision,avg_accuracy,avg_accuracy_base,avg_tcr)
df_j = ...

df_table = pd.concat([df_i, df_j])
df_table.set_index('#', inplace=True) #para matangal yung index na paulit ulit
df_table = df_table.rename_axis(None) #para matangal si #
df_table

#Creating a Scatter Plot
#define colors if you want
colors = ['b', 'r', 'g', 'm']

#You can plot using 2 methods
#1.Direct 
plt.scatter(x_coord, y_coord, marker='o', color=colors[0])
#or
plt.scatter(x_coord_list, y_coord_list, marker='o', color=colors[0])
#2 Indirect with legend
scatter_1 = plt.scatter(x_coord, y_coord, marker='o', color=colors[0])
plt.legend((scatter_1,...),
           ('no lemmatizer, no stop-list', 'no lemmatizer, top-100 stop-list', 'with lemmatizer, no stop-list', 'with lemmatizer, top-100 stop-list'),
           scatterpoints=1,
           loc='lower right', #where the legend box will be located
           ncol=1, 
           fontsize=8 #font size of legend)

plt.xlabel('spam recall') #x field
plt.ylabel('spam precision') #y field

plt.grid() #if you want to show grid
plt.show() #display

#For Line Scatter Plot
#Use this to connect the lines of the same type
plt.plot(x_coord, y_coord, ls = "dotted", c=colors[3])
#or
plt.plot(x_coord_list, y_coord_list, ls = "dotted", c=colors[3])

#REMEMBER you can simply replace the x_coord or y_coord with a LIST, Python does the rest
